# 10.1 応用
・量子シミュレーションと化学領域\
・線形代数のスピードアップ\
・最適化\
・テンソルネットワーク\
など

# 10.3 将来展望
量子エラー訂正は量子コンピューターの研究の中でも労力が割かれている部分である。
Surface_code, McCleanによるsubspace expansionによるエラー緩和（VQEの章）、アンチドジッター理論/共形場理論対応から派生したエラー訂正理論などが言及されている。アンチドジッター理論/共形場理論対応は宇宙論と素粒子論・量子論との対応である。その意味で、量子コンピュータへの直接的な応用がなくても物理の研究として活用できる場合もある。

# 10.2 量子超越性
量子超越性とは、最新の古典コンピュータでも効率的に解けない問題を量子コンピュータが効率よく解けることを指す。
ただし、その問題が実用的かどうかは問わない。
量子超越性を示す実験は、（ベルの不等式の破れのように）これまで一般に受け入れられた仮説を覆すことがある。
その一つとして、全てのアルゴリズムは確率的チューリングマシンで効率的にシミュレートできるという仮説(Extended Church-Turing Thesis)が、量子エラー訂正の実験によって破られた例がある。（４章で議論）

## ランダム回路でのサンプリング問題
量子超越性を考える一つの単純な例として、古典的なメモリの表現力がある。
nキュービットの量子系は$2^n$の係数を持ち、その振幅を表すのに8バイト必要、などを考えると現状のRAMなどで扱えないキュービット数などを計算することはできる。
キュービット数が増えると、古典的枠組みにおいて（物理的もしくは時間的に）リソースが指数関数的に増える、というのがランダム回路の量子超越性においての根本的なアイデアである。
以下の論文の内容に沿って量子超越性について議論する。
"Characterizing Quantum Supremacy in Near-Term Devices" https://arxiv.org/pdf/1608.00263.pdf

まず、ランダム回路を以下のように定義する。\
※ランダム回路とは、出力パターンが均一にランダムになるものではなく、ある出力パターン{$x_j$}$_{j=1..N}$（$N=2^n$）が実現する確率が$p$のとき、同じ$p$の確率で出現する出力パターンの割合が$e^{-Np}$のように分布（Porter-Thomas分布）する量子回路のことを言う。
以下の回路はその性質を満たす。
1. アダマールゲートを全キュービットに作用
2. 隣接するキュービットについて、Cnotrolled_Zゲートを気まぐれに作用させる。（作用させなくてもよい）
3. 以下の基準で{$X^{1/2}$,$Y^{1/2}$,$T$}を作用させる。($T=[[1,0], [0, e^{i \pi/4}]]$)\
 ・前のサイクル（回路でいう一つ左）でCZゲートが使われていれば、$X^{1/2}$または$Y^{1/2}$を作用。\
 ・前のサイクルで$X^{1/2}$または$Y^{1/2}$が使われていれば、{$X^{1/2}$,$Y^{1/2}$,$T$}（ただし2.でCZが作用させられていない場合に限る）\
 ・これまでのサイクルで{$X^{1/2}$,$Y^{1/2}$,$T$}が使われれいなければ$T$を作用。\
 ・上のどれも満たされていなければ何もしない。
4. 2.と3.を決められたサイクルだけ繰り返す。
5. Z基底かX基底で測定する

実際に回路を生成するコードの詳細はこのページ最下段のAppendix参照。ここでは4×4の二次元格子に並んだキュービットを考え、Appendix中のgenerate_boixo_2018_supremacy_circuits_v2_grid
メソッドを実行する。

In [4]:
#Number of rows in grid of quibts
n_rows = 4

#Number of columns in grid of qubits
n_cols = 4

# Depth of CZ gates in supremacy circuit
cz_depth = 5

#Generate  the supremacy circuit
generate_boixo_2018_supremacy_circuits_v2_grid(n_rows, n_cols, cz_depth, seed=123)

┌───────────────┐           ┌──────────┐
(0, 0): ───H───@────X^0.5──────────────T────────@─────────────X^0.5───H───
               │                                │
(0, 1): ───H───@────X^0.5──────────────@────────┼────X^0.5────T───────H───
                                       │        │
(0, 2): ───H───T───────────────────────@────────┼────@────────@───────H───
                                                │    │        │
(0, 3): ───H───T────────────────────────────────┼────┼────────@───────H───
                                                │    │
(1, 0): ───H───T─────────@─────────────Y^0.5────@────┼────────@───────H───
                         │                           │        │
(1, 1): ───H───T─────────┼───────────────────────────┼────────@───────H───
                         │                           │
(1, 2): ───H───@────@────┼─────────────X^0.5─────────@────────X^0.5───H───
               │    │    │
(1, 3): ───H───@────┼────┼────X^0.5────T──────────────────────────────H───
                    │    │
(2, 0): ───H───@────┼────@─────────────Y^0.5────T─────────────────────H───
               │    │
(2, 1): ───H───@────┼────X^0.5─────────@─────────────@────────X^0.5───H───
                    │                  │             │
(2, 2): ───H───T────@──────────────────@────────X^0.5┼────────@───────H───
                                                     │        │
(2, 3): ───H───T────────────────────────────────@────┼────────@───────H───
                                                │    │
(3, 0): ───H───T────────────────────────────────┼────┼────────@───────H───
                                                │    │        │
(3, 1): ───H───T────────────────────────────────┼────@────────@───────H───
                                                │
(3, 2): ───H───@────Y^0.5──────────────T────────┼─────────────────────H───
               │                                │
(3, 3): ───H───@────X^0.5──────────────T────────@─────────────X^0.5───H───
                   └───────────────┘           └──────────┘

この回路を用いて量子超越性を確かめる実験は以下の通り。
1. キュービット数$n$で深さ$d$のランダム回路$U$を作る。
2. 回路の結果を$m \sim10^3-10^6$回出力する。結果を{$\vec{x}_j$}$_{j=1..m}$とする。
3. $1/p_U (\vec{x}_j)$を古典計算機で求める。ただし$|\psi \rangle = U |{\psi_0}\rangle $を回路の終状態として
\begin{equation}
p_U (\vec{x}_j) := |\langle \vec{x}_j | \psi \rangle | ^2
\end{equation}
4. 以下の値を計算
\begin{equation}
\alpha = H_0 - \frac{1}{m} \sum_{j=1}^{m} {\rm log} \frac{1}{p_U (\vec{x}_j) }
\end{equation}
ここで、$H_0 = {\rm log} (2^n) + \gamma$  ($\gamma \sim 0.577$はオイラー定数)は出力が均一ランダムな回路があったときの交差エントロピー。
5. 後述する$C$を用いて以下が成立していれば量子超越性があると言える。
\begin{equation}
C \leq \alpha  \leq 1
\end{equation}

次に$\alpha$と$C$の意味について考える。$p(\vec{x}_j )$を$\vec{x}_j $に対する任意の確率分布として、$\Delta H(p)$を
\begin{equation}
\Delta H(p) := H_0 - H(p, p_U)
\end{equation}
と定義する。ただし$H(p, p_U)　 =  -\sum p {\rm log} p_U$は交差エントロピーで$p$と$p_U$が近いほど小さくなる。逆に、$\Delta H(p)$は$p$と$p_U$の近さを評価する値となる。これを用いて、$\alpha$と$C$は以下のように表される。

\begin{equation}
\alpha = E_U(\Delta H(p_{exp}) ),~~~C = E_U(\Delta H(p_A) )
\end{equation}

ただし$E_U$は$U$に関するアンサンブル期待値である。$p_{exp}$は実際に量子回路を動かして動かして得られた確率分布で、$\kappa_U$をノイズ含めた回路の写像とすると、$p_{exp}(\vec{x}_j|U) = \langle \vec{x}_j |\kappa_U(| \psi_0 \rangle \langle \psi_0| ) | \vec{x}_j  \rangle$と表される。$p_A$は現存する古典的コンピュータで最良のアルゴリズムを多項式時間使った場合に得られる出力の確率分布である。つまり、理想的な量子ランダム回路の出力$p_U$に対して、$\alpha$はノイズ含めた量子回路の出力分布がどの程度$p_U$に近いかを表し、$C$は現存の古典コンピュータが最善を尽くすとどの程度$p_U$を予測できるかを表している。これより、$C\leq \alpha$であれば量子超越性がある、と理解できる。

回路のサイズが大きくなれば、古典計算は難しくなり、$C$が0に近づくため量子超越性は見かけ上実現しやすくなる。
しかし、回路のサイズが大きくなればノイズが大きくなり、さらに$p_U$を求めるための統計的なサンプル出力が多く必要になるため、$p_{exp}$から$p_U$を求めるのが難しくなっていく。つまり$\alpha$を求めるのが難しくなっていく。この求め方は論文参照。https://arxiv.org/pdf/1608.00263.pdf

（呟き）ちなみに、$\alpha$が4.の形にかけるのは、$U$に関するアンサンブルを取っているからとのこと。1/mはわかるが、log(1/p_U)の部分はそのままでよいのか？

## Appendix: Boxioランダム回路生成のためのコード
以下のページのコピペ
https://cirq.readthedocs.io/en/stable/_modules/cirq/experiments/google_v2_supremacy_circuit.html

In [2]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import random
from typing import Callable, Iterable, TypeVar, cast, Sequence

from cirq.circuits import InsertStrategy
from cirq import circuits, devices, google, ops


def generate_boixo_2018_supremacy_circuits_v2(
        qubits: Iterable[devices.GridQubit], cz_depth: int,
        seed: int) -> circuits.Circuit:
    """
    Generates Google Random Circuits v2 as in github.com/sboixo/GRCS cz_v2.
    See also https://arxiv.org/abs/1807.10749

    Args:
        qubits: qubit grid in which to generate the circuit.
        cz_depth: number of layers with CZ gates.
        seed: seed for the random instance.

    Returns:
        A circuit corresponding to instance
        inst_{n_rows}x{n_cols}_{cz_depth+1}_{seed}

    The mapping of qubits is cirq.GridQubit(j,k) -> q[j*n_cols+k]
    (as in the QASM mapping)
    """

    non_diagonal_gates = [ops.pauli_gates.X**(1/2), ops.pauli_gates.Y**(1/2)]
    rand_gen = random.Random(seed).random

    circuit = circuits.Circuit()

    # Add an initial moment of Hadamards
    circuit.append(ops.common_gates.H(qubit) for qubit in qubits)

    layer_index = 0
    if cz_depth:
        layer_index = _add_cz_layer(layer_index, circuit)
        # In the first moment, add T gates when possible
        for qubit in qubits:
            if not circuit.operation_at(qubit, 1):
                circuit.append(ops.common_gates.T(qubit),
                               strategy=InsertStrategy.EARLIEST)

    for moment_index in range(2, cz_depth+1):
        layer_index = _add_cz_layer(layer_index, circuit)
        # Add single qubit gates in the same moment
        for qubit in qubits:
            if not circuit.operation_at(qubit, moment_index):
                last_op = circuit.operation_at(qubit, moment_index-1)
                if last_op:
                    gate = cast(ops.GateOperation, last_op).gate
                    # Add a random non diagonal gate after a CZ
                    if gate == ops.CZ:
                        circuit.append(_choice(rand_gen,
                                               non_diagonal_gates).on(qubit),
                                       strategy=InsertStrategy.EARLIEST)
                    # Add a T gate after a non diagonal gate
                    elif not gate == ops.T:
                        circuit.append(ops.common_gates.T(qubit),
                                       strategy=InsertStrategy.EARLIEST)

    # Add a final moment of Hadamards
    circuit.append([ops.common_gates.H(qubit) for qubit in qubits],
                   strategy=InsertStrategy.NEW_THEN_INLINE)

    return circuit



def generate_boixo_2018_supremacy_circuits_v2_grid(n_rows: int, n_cols: int,
                                                   cz_depth: int, seed: int
                                                  ) -> circuits.Circuit:
    """
    Generates Google Random Circuits v2 as in github.com/sboixo/GRCS cz_v2.
    See also https://arxiv.org/abs/1807.10749

    Args:
        n_rows: number of rows of a 2D lattice.
        n_cols: number of columns.
        cz_depth: number of layers with CZ gates.
        seed: seed for the random instance.

    Returns:
        A circuit corresponding to instance
        inst_{n_rows}x{n_cols}_{cz_depth+1}_{seed}

    The mapping of qubits is cirq.GridQubit(j,k) -> q[j*n_cols+k]
    (as in the QASM mapping)
    """
    qubits = [devices.GridQubit(i, j) for i in range(n_rows)
              for j in range(n_cols)]
    return generate_boixo_2018_supremacy_circuits_v2(qubits, cz_depth, seed)



def generate_boixo_2018_supremacy_circuits_v2_bristlecone(
        n_rows: int, cz_depth: int, seed: int) -> circuits.Circuit:
    """
    Generates Google Random Circuits v2 in Bristlecone.
    See also https://arxiv.org/abs/1807.10749

    Args:
        n_rows: number of rows in a Bristlecone lattice.
          Note that we do not include single qubit corners.
        cz_depth: number of layers with CZ gates.
        seed: seed for the random instance.

    Returns:
        A circuit with given size and seed.
    """
    def get_qubits(n_rows):
        def count_neighbors(qubits, qubit):
            """Counts the qubits that the given qubit can interact with."""
            possibles = [
                devices.GridQubit(qubit.row + 1, qubit.col),
                devices.GridQubit(qubit.row - 1, qubit.col),
                devices.GridQubit(qubit.row, qubit.col + 1),
                devices.GridQubit(qubit.row, qubit.col - 1),
                ]
            return len(list(e for e in possibles if e in qubits))

        assert 2 <= n_rows <= 11
        max_row = n_rows - 1
        dev = google.Bristlecone
        # we need a consistent order of qubits
        qubits = list(dev.qubits)
        qubits.sort()
        qubits = [q for q in qubits
                      if  q.row <= max_row and  q.row + q.col < n_rows + 6
                      and q.row - q.col < n_rows - 5]
        qubits = [q for q in qubits if count_neighbors(qubits, q) > 1]
        return qubits

    qubits = get_qubits(n_rows)
    return generate_boixo_2018_supremacy_circuits_v2(qubits, cz_depth, seed)



T = TypeVar('T')


def _choice(rand_gen: Callable[[], float], sequence: Sequence[T]) -> T:
    """Choose a random element from a non-empty sequence.

    Use this instead of random.choice, with random.random(), for reproducibility
    """
    return sequence[int(rand_gen() * len(sequence))]


def _add_cz_layer(layer_index: int, circuit: circuits.Circuit) -> int:
    cz_layer = None
    while not cz_layer:
        qubits = cast(Iterable[devices.GridQubit], circuit.all_qubits())
        cz_layer = list(_make_cz_layer(qubits, layer_index))
        layer_index += 1

    circuit.append(cz_layer, strategy=InsertStrategy.NEW_THEN_INLINE)
    return layer_index


def _make_cz_layer(qubits: Iterable[devices.GridQubit], layer_index: int
                   ) -> Iterable[ops.Operation]:
    """
    Each layer index corresponds to a shift/transpose of this CZ pattern:

        ●───●   ●   ●   ●───●   ●   ● . . .

        ●   ●   ●───●   ●   ●   ●───● . . .

        ●───●   ●   ●   ●───●   ●   ● . . .

        ●   ●   ●───●   ●   ●   ●───● . . .

        ●───●   ●   ●   ●───●   ●   ● . . .

        ●   ●   ●───●   ●   ●   ●───● . . .
        .   .   .   .   .   .   .   . .
        .   .   .   .   .   .   .   .   .
        .   .   .   .   .   .   .   .     .

    Labelled edges, showing the exact index-to-CZs mapping (mod 8):

         ●─0─●─2─●─4─●─6─●─0─. . .
        3│  7│  3│  7│  3│
         ●─4─●─6─●─0─●─2─●─4─. . .
        1│  5│  1│  5│  1│
         ●─0─●─2─●─4─●─6─●─0─. . .
        7│  3│  7│  3│  7│
         ●─4─●─6─●─0─●─2─●─4─. . .
        5│  1│  5│  1│  5│
         ●─0─●─2─●─4─●─6─●─0─. . .
        3│  7│  3│  7│  3│
         .   .   .   .   .   .
         .   .   .   .   .     .
         .   .   .   .   .       .

    Note that, for small devices, some layers will be empty because the layer
    only contains edges not present on the device.
    """

    # map to an internal layer index to match the cycle order of public circuits
    layer_index_map = [0, 3, 2, 1, 4, 7, 6, 5]
    internal_layer_index = layer_index_map[layer_index % 8]

    dir_row = internal_layer_index % 2
    dir_col = 1 - dir_row
    shift = (internal_layer_index >> 1) % 4

    for q in qubits:
        q2 = devices.GridQubit(q.row + dir_row, q.col + dir_col)
        if q2 not in qubits:
            continue  # This edge isn't on the device.
        if (q.row * (2 - dir_row) + q.col * (2 - dir_col)) % 4 != shift:
            continue  # No CZ along this edge for this layer.

        yield ops.common_gates.CZ(q, q2)